In [25]:
import os
import numpy as np
import pandas as pd
from utils import *

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.metrics import euclidean_distances
from sklearn.pipeline import make_pipeline

In [2]:
path_to_data = '../data'
path_track = os.path.join(path_to_data, '20210824_212829_tracks.tsv')
path_playlist = os.path.join(path_to_data, '20210824_212829_playlists.tsv')
path_audio =  os.path.join(path_to_data, '20210824_212829_audios.tsv')
path_artist =  os.path.join(path_to_data, '20210824_212829_artists.tsv')

In [3]:
pd_track = pd.read_csv(path_track, sep='\t')
pd_audio = pd.read_csv(path_audio, sep='\t')
pd_playlist = pd.read_csv(path_playlist, sep='\t')
pd_artist = pd.read_csv(path_artist, sep='\t')

In [4]:
pd_audio

,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,5HCyWlXZPP0y6Gqq8TgA20,0.591,0.764,1,-5.484,1,0.0483,0.0383,0.000000,0.1030,0.478,169.928,4
1,27NovPIUIRrOZoCHxABJwK,0.736,0.704,3,-7.409,0,0.0615,0.0203,0.000000,0.0501,0.894,149.995,4
2,3Vi5XqYrmQgOYBajMWSvCi,0.664,0.609,1,-6.509,1,0.0707,0.3040,0.000000,0.0926,0.194,130.041,4
3,6PQ88X9TkUIAUIZJHW2upE,0.808,0.897,11,-3.712,0,0.0348,0.0469,0.000031,0.3640,0.591,126.026,4
4,3Wrjm47oTz2sjIgck11l5e,0.714,0.800,11,-4.808,0,0.0504,0.1270,0.000000,0.3590,0.589,134.002,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69866,3Zuf70897YkrVRAsrBMMSF,0.751,0.753,0,-8.198,1,0.0935,0.2270,0.000784,0.2090,0.774,106.976,4
69867,1MDoll6jK4rrk2BcFRP5i7,0.471,0.431,5,-6.129,0,0.0342,0.3290,0.000000,0.0854,0.289,157.980,4
69868,7DqTaelFf846rl1CHmyfOW,0.506,0.433,2,-14.183,0,0.0345,0.8420,0.031100,0.1120,0.641,83.222,4
69869,0ygTmpa6uSotkBkTiwcMZ4,0.493,0.590,0,-5.814,0,0.0392,0.3390,0.000000,0.1210,0.502,169.908,4


In [5]:
pd_audio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69871 entries, 0 to 69870
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                69871 non-null  object 
 1   danceability      69871 non-null  float64
 2   energy            69871 non-null  float64
 3   key               69871 non-null  int64  
 4   loudness          69871 non-null  float64
 5   mode              69871 non-null  int64  
 6   speechiness       69871 non-null  float64
 7   acousticness      69871 non-null  float64
 8   instrumentalness  69871 non-null  float64
 9   liveness          69871 non-null  float64
 10  valence           69871 non-null  float64
 11  tempo             69871 non-null  float64
 12  time_signature    69871 non-null  int64  
dtypes: float64(9), int64(3), object(1)
memory usage: 6.9+ MB


In [6]:
pd_track

,playlist_id,track_id,track_name,track_duration,artist_ids,artist_names,album_id,album_name,popularity
0,37i9dQZF1DXcBWIGoYBM5M,5HCyWlXZPP0y6Gqq8TgA20,STAY (with Justin Bieber),141805,"['2tIP7SsRs7vjIcLrU85W8J', '1uNFoZAHBGtllmzznp...","['The Kid LAROI', 'Justin Bieber']",4QLAtpLNUsHEYrcHXmMIZZ,STAY (with Justin Bieber),100
1,37i9dQZF1DXcBWIGoYBM5M,27NovPIUIRrOZoCHxABJwK,INDUSTRY BABY (feat. Jack Harlow),212000,"['7jVv8c5Fj3E9VhNjxT4snq', '2LIk90788K0zvyj2JJ...","['Lil Nas X', 'Jack Harlow']",622NFw5Yk0OReMJ2XWcXUh,INDUSTRY BABY (feat. Jack Harlow),97
2,37i9dQZF1DXcBWIGoYBM5M,3Vi5XqYrmQgOYBajMWSvCi,Need To Know,210560,['5cj0lLjcoR7YOSnhnX0Po5'],['Doja Cat'],1nAQbHeOWTfQzbOoFrvndW,Planet Her,92
3,37i9dQZF1DXcBWIGoYBM5M,6PQ88X9TkUIAUIZJHW2upE,Bad Habits,231041,['6eUKZXaKkcviH0Ku9w2n3V'],['Ed Sheeran'],01NhUvviMytvV12pmJiDZH,Bad Habits,98
4,37i9dQZF1DXcBWIGoYBM5M,3Wrjm47oTz2sjIgck11l5e,Beggin',211560,['0lAWpj5szCSwM4rUMHYmrr'],['Måneskin'],2qJw6w5XwQO0PQlSWPu7Tw,Chosen,99
...,...,...,...,...,...,...,...,...,...
88814,4lgseztVwmKQ8MNETPVIny,5kqIPrATaCc2LqxVWzQGbk,7 Years,237300,['25u4wHJWxCA9vO0CzxAbK7'],['Lukas Graham'],4rFrdkSWs0dtj0rWPzOk1v,Lukas Graham,79
88815,4lgseztVwmKQ8MNETPVIny,3FCto7hnn1shUyZL42YgfO,Piano Man,336093,['6zFYqv1mOsgBRQbae3JJ9e'],['Billy Joel'],7r36rel1M4gyBavfcJP6Yz,The Essential Billy Joel,66
88816,4lgseztVwmKQ8MNETPVIny,3XVBdLihbNbxUwZosxcGuJ,If I Ain't Got You,228706,['3DiDSECUqqY1AuBP8qtaIa'],['Alicia Keys'],6TqRKHLjDu5QZuC8u5Woij,The Diary Of Alicia Keys,78
88817,4lgseztVwmKQ8MNETPVIny,6QPKYGnAW9QozVz2dSWqRg,Someone Like You,285240,['4dpARuHxo51G3z768sgnrY'],['Adele'],7n3QJc7TBOxXtlYh4Ssll8,21,13


In [7]:
pd_track.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88819 entries, 0 to 88818
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   playlist_id     88819 non-null  object
 1   track_id        88819 non-null  object
 2   track_name      88812 non-null  object
 3   track_duration  88819 non-null  int64 
 4   artist_ids      88819 non-null  object
 5   artist_names    88819 non-null  object
 6   album_id        88819 non-null  object
 7   album_name      88811 non-null  object
 8   popularity      88819 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 6.1+ MB


In [8]:
pd_playlist

,playlist_id,playlist_name,playlist_description,playlist_num_tracks,playlist_num_followers
0,37i9dQZF1DXcBWIGoYBM5M,Today's Top Hits,Doja Cat is on top of the Hottest 50!,50,29379379
1,37i9dQZF1DX0XUsuxWHRQd,RapCaviar,"New music from Young Thug, Trippie Redd, Rod W...",50,14030878
2,37i9dQZF1DX1lVhptIYRda,Hot Country,"Today's top country hits of the week, worldwid...",53,6475778
3,37i9dQZF1DX10zKzsJ2jva,¡Viva Latino!,"Today's top Latin hits are right here, on ¡Viv...",50,11218997
4,37i9dQZF1DX4JAvHpjipBk,New Music Friday,"New music from Lorde, Young Thug, Ed Sheeran, ...",100,3788318
...,...,...,...,...,...
1390,37i9dQZF1DX3WvGXE8FqYX,Women of Pop,Kick back with the queens of pop.,70,2185426
1391,5ji4GZJpll6twskFvKxiHx,dw-c,NaN,50,14
1392,40VxbK9NqccdUDUpiUXmbp,dw_g,NaN,30,8
1393,0RTz1jFo5BXGPfI8eVf8sj,Top Shower Songs,NaN,100,57


In [9]:
pd_playlist.fillna('None')

,playlist_id,playlist_name,playlist_description,playlist_num_tracks,playlist_num_followers
0,37i9dQZF1DXcBWIGoYBM5M,Today's Top Hits,Doja Cat is on top of the Hottest 50!,50,29379379
1,37i9dQZF1DX0XUsuxWHRQd,RapCaviar,"New music from Young Thug, Trippie Redd, Rod W...",50,14030878
2,37i9dQZF1DX1lVhptIYRda,Hot Country,"Today's top country hits of the week, worldwid...",53,6475778
3,37i9dQZF1DX10zKzsJ2jva,¡Viva Latino!,"Today's top Latin hits are right here, on ¡Viv...",50,11218997
4,37i9dQZF1DX4JAvHpjipBk,New Music Friday,"New music from Lorde, Young Thug, Ed Sheeran, ...",100,3788318
...,...,...,...,...,...
1390,37i9dQZF1DX3WvGXE8FqYX,Women of Pop,Kick back with the queens of pop.,70,2185426
1391,5ji4GZJpll6twskFvKxiHx,dw-c,None,50,14
1392,40VxbK9NqccdUDUpiUXmbp,dw_g,None,30,8
1393,0RTz1jFo5BXGPfI8eVf8sj,Top Shower Songs,None,100,57


In [10]:
pd_playlist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1395 entries, 0 to 1394
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   playlist_id             1395 non-null   object
 1   playlist_name           1395 non-null   object
 2   playlist_description    1239 non-null   object
 3   playlist_num_tracks     1395 non-null   int64 
 4   playlist_num_followers  1395 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 54.6+ KB


In [11]:
pd_artist

,id,name,followers,popularity,genres
0,2xgAJkalFHfceCNGETOkyM,GRX,76585,43,[]
1,6enyYil5r9snkIazgAX5DG,Jo Ann Kelly,998,20,['british blues']
2,40oYPr305MsT2lsiXr9fX9,The Boomtown Rats,116274,54,"['new romantic', 'new wave', 'new wave pop', '..."
3,3i8iJVU0mtgzbZsuF1AoJ3,sad alex,23548,52,"['alt z', 'modern indie pop']"
4,5oFkj1qSlyBUmV5d6Edgtq,Cozz,146831,57,['underground hip hop']
...,...,...,...,...,...
34649,1tY6Z4NEPI3xHhVA8VU9Jl,Vishwa Mohan Bhatt,23094,30,"['hindustani classical', 'hindustani instrumen..."
34650,7lPv78tlT005m22dYvSdNg,Lindsey Leblanc,52,32,[]
34651,6iXZPP5qLB2dtIQ4WuNk9M,Thelonious Coltrane,4359,44,"['chillhop', 'jazz boom bap', 'lo-fi jazzhop']"
34652,7s9LTAKRwkBQ0CqWlbE254,Frase,2245,36,[]


In [12]:
pd_artist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34654 entries, 0 to 34653
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          34654 non-null  object
 1   name        34654 non-null  object
 2   followers   34654 non-null  int64 
 3   popularity  34654 non-null  int64 
 4   genres      34654 non-null  object
dtypes: int64(2), object(3)
memory usage: 1.3+ MB


## Mô tả dữ liệu

+ Dataframe `pd_track` mô tả thông tin về một playlist sẽ chứa các track nào, một `playlist_id` sẽ có thể có nhiều `track_id`. Ngoài ra còn một số thông tin khác ở đây chẳng hạn như độ phổ biến, thời lượng track,...
+ Sử dụng các `track_id` để truy xuất các thông tin đặc trưng về âm thanh tại dataframe `pd_audio`. Tại Dataframe này, các đặc trưng âm học sẽ được biểu diễn, đây sẽ là dataframe chúng ta sẽ thực hiện các công việc chuẩn hóa và tính toán độ dị biệt giữa các track

In [13]:
pd_track = pd.merge(pd_track, pd_audio, left_on = 'track_id', right_on = 'id')
pd_track

,playlist_id,track_id,track_name,track_duration,artist_ids,artist_names,album_id,album_name,popularity,id,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,37i9dQZF1DXcBWIGoYBM5M,5HCyWlXZPP0y6Gqq8TgA20,STAY (with Justin Bieber),141805,"['2tIP7SsRs7vjIcLrU85W8J', '1uNFoZAHBGtllmzznp...","['The Kid LAROI', 'Justin Bieber']",4QLAtpLNUsHEYrcHXmMIZZ,STAY (with Justin Bieber),100,5HCyWlXZPP0y6Gqq8TgA20,...,1,-5.484,1,0.0483,0.0383,0.000000,0.1030,0.478,169.928,4
1,37i9dQZF1DXcRXFNfZr7Tp,5HCyWlXZPP0y6Gqq8TgA20,STAY (with Justin Bieber),141805,"['2tIP7SsRs7vjIcLrU85W8J', '1uNFoZAHBGtllmzznp...","['The Kid LAROI', 'Justin Bieber']",4QLAtpLNUsHEYrcHXmMIZZ,STAY (with Justin Bieber),100,5HCyWlXZPP0y6Gqq8TgA20,...,1,-5.484,1,0.0483,0.0383,0.000000,0.1030,0.478,169.928,4
2,37i9dQZF1DXc2aPBXGmXrt,5HCyWlXZPP0y6Gqq8TgA20,STAY (with Justin Bieber),141805,"['2tIP7SsRs7vjIcLrU85W8J', '1uNFoZAHBGtllmzznp...","['The Kid LAROI', 'Justin Bieber']",4QLAtpLNUsHEYrcHXmMIZZ,STAY (with Justin Bieber),100,5HCyWlXZPP0y6Gqq8TgA20,...,1,-5.484,1,0.0483,0.0383,0.000000,0.1030,0.478,169.928,4
3,37i9dQZF1DWYD2pxgooM7y,5HCyWlXZPP0y6Gqq8TgA20,STAY (with Justin Bieber),141805,"['2tIP7SsRs7vjIcLrU85W8J', '1uNFoZAHBGtllmzznp...","['The Kid LAROI', 'Justin Bieber']",4QLAtpLNUsHEYrcHXmMIZZ,STAY (with Justin Bieber),100,5HCyWlXZPP0y6Gqq8TgA20,...,1,-5.484,1,0.0483,0.0383,0.000000,0.1030,0.478,169.928,4
4,37i9dQZF1DWUa8ZRTfalHk,5HCyWlXZPP0y6Gqq8TgA20,STAY (with Justin Bieber),141805,"['2tIP7SsRs7vjIcLrU85W8J', '1uNFoZAHBGtllmzznp...","['The Kid LAROI', 'Justin Bieber']",4QLAtpLNUsHEYrcHXmMIZZ,STAY (with Justin Bieber),100,5HCyWlXZPP0y6Gqq8TgA20,...,1,-5.484,1,0.0483,0.0383,0.000000,0.1030,0.478,169.928,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88618,0RTz1jFo5BXGPfI8eVf8sj,3Zuf70897YkrVRAsrBMMSF,"You're The One That I Want - From ""Grease"" Ori...",168466,"['4hKkEHkaqCsyxNxXEsszVH', '4BoRxUdrcgbbq1rxJv...","['John Travolta', 'Olivia Newton-John']",0CbwlsDekgmAW03uMBjPBG,Grease (Deluxe Edition),0,3Zuf70897YkrVRAsrBMMSF,...,0,-8.198,1,0.0935,0.2270,0.000784,0.2090,0.774,106.976,4
88619,4lgseztVwmKQ8MNETPVIny,1MDoll6jK4rrk2BcFRP5i7,Hello,295502,['4dpARuHxo51G3z768sgnrY'],['Adele'],5y28RdPzIRXegzr5YM9Y0D,Hello,3,1MDoll6jK4rrk2BcFRP5i7,...,5,-6.129,0,0.0342,0.3290,0.000000,0.0854,0.289,157.980,4
88620,4lgseztVwmKQ8MNETPVIny,7DqTaelFf846rl1CHmyfOW,Chan Chan,257737,['11kBu957KTYoAltZHDm8gW'],['Buena Vista Social Club'],2Upqk0mMh9OMIVSj9F8Xzw,Buena Vista Social Club,0,7DqTaelFf846rl1CHmyfOW,...,2,-14.183,0,0.0345,0.8420,0.031100,0.1120,0.641,83.222,4
88621,4lgseztVwmKQ8MNETPVIny,0ygTmpa6uSotkBkTiwcMZ4,Warwick Avenue,226386,['37NqXwtb6nIEqRt4EJSoIO'],['Duffy'],4x0HJqUUDRvJLZbie95Ayk,Rockferry (Deluxe Edition),0,0ygTmpa6uSotkBkTiwcMZ4,...,0,-5.814,0,0.0392,0.3390,0.000000,0.1210,0.502,169.908,4


In [14]:
pd_full = pd.merge(pd_track, pd_playlist, left_on = 'playlist_id', right_on = 'playlist_id')
pd.set_option('display.max_columns', None)
pd_full

,playlist_id,track_id,track_name,track_duration,artist_ids,artist_names,album_id,album_name,popularity,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,playlist_name,playlist_description,playlist_num_tracks,playlist_num_followers
0,37i9dQZF1DXcBWIGoYBM5M,5HCyWlXZPP0y6Gqq8TgA20,STAY (with Justin Bieber),141805,"['2tIP7SsRs7vjIcLrU85W8J', '1uNFoZAHBGtllmzznp...","['The Kid LAROI', 'Justin Bieber']",4QLAtpLNUsHEYrcHXmMIZZ,STAY (with Justin Bieber),100,5HCyWlXZPP0y6Gqq8TgA20,0.591,0.764,1,-5.484,1,0.0483,0.038300,0.000000,0.1030,0.478,169.928,4,Today's Top Hits,Doja Cat is on top of the Hottest 50!,50,29379379
1,37i9dQZF1DXcBWIGoYBM5M,27NovPIUIRrOZoCHxABJwK,INDUSTRY BABY (feat. Jack Harlow),212000,"['7jVv8c5Fj3E9VhNjxT4snq', '2LIk90788K0zvyj2JJ...","['Lil Nas X', 'Jack Harlow']",622NFw5Yk0OReMJ2XWcXUh,INDUSTRY BABY (feat. Jack Harlow),97,27NovPIUIRrOZoCHxABJwK,0.736,0.704,3,-7.409,0,0.0615,0.020300,0.000000,0.0501,0.894,149.995,4,Today's Top Hits,Doja Cat is on top of the Hottest 50!,50,29379379
2,37i9dQZF1DXcBWIGoYBM5M,3Vi5XqYrmQgOYBajMWSvCi,Need To Know,210560,['5cj0lLjcoR7YOSnhnX0Po5'],['Doja Cat'],1nAQbHeOWTfQzbOoFrvndW,Planet Her,92,3Vi5XqYrmQgOYBajMWSvCi,0.664,0.609,1,-6.509,1,0.0707,0.304000,0.000000,0.0926,0.194,130.041,4,Today's Top Hits,Doja Cat is on top of the Hottest 50!,50,29379379
3,37i9dQZF1DXcBWIGoYBM5M,6PQ88X9TkUIAUIZJHW2upE,Bad Habits,231041,['6eUKZXaKkcviH0Ku9w2n3V'],['Ed Sheeran'],01NhUvviMytvV12pmJiDZH,Bad Habits,98,6PQ88X9TkUIAUIZJHW2upE,0.808,0.897,11,-3.712,0,0.0348,0.046900,0.000031,0.3640,0.591,126.026,4,Today's Top Hits,Doja Cat is on top of the Hottest 50!,50,29379379
4,37i9dQZF1DXcBWIGoYBM5M,3Wrjm47oTz2sjIgck11l5e,Beggin',211560,['0lAWpj5szCSwM4rUMHYmrr'],['Måneskin'],2qJw6w5XwQO0PQlSWPu7Tw,Chosen,99,3Wrjm47oTz2sjIgck11l5e,0.714,0.800,11,-4.808,0,0.0504,0.127000,0.000000,0.3590,0.589,134.002,4,Today's Top Hits,Doja Cat is on top of the Hottest 50!,50,29379379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88618,3yUO32lGCEnAfFGORSEsy4,6jKNYORXeWGGGiSkj1sBZu,Cocky Tuff Up,170111,['5mYWkDD4b1eM4ZjFq5axxs'],['Aidonia'],4iSfhNdg4CcSoHBZ8MNR0S,Matte Black Riddim,0,6jKNYORXeWGGGiSkj1sBZu,0.776,0.924,7,-4.651,1,0.0580,0.243000,0.000075,0.1580,0.749,104.975,4,Raw Expression,Brukout edition: It's a raw and sexy dancehall...,42,307
88619,3yUO32lGCEnAfFGORSEsy4,5DycXVa4e82E6k9M15Nf51,Turn It Up (feat. DJ Jazzy T),220978,"['1RnGhd2JfN5nbVOvYmhDyO', '4DxQlFDzfkQjyU9mrq...","[""Ce'Cile"", 'DJ Jazzy T']",0ZEmsBG911uKT7CJxlF0DS,Still Running,6,5DycXVa4e82E6k9M15Nf51,0.646,0.947,7,-3.631,1,0.0772,0.000389,0.000057,0.8960,0.793,100.027,4,Raw Expression,Brukout edition: It's a raw and sexy dancehall...,42,307
88620,3yUO32lGCEnAfFGORSEsy4,41gA2hLHRJW4bC40lm2X7R,Cup Yah Full,227552,['21J3YJTyq1biE3SvSNjzuf'],['Capleton'],1GIhVkizUAMw7Zci6DMPdG,Cup Yah Full - Single,0,41gA2hLHRJW4bC40lm2X7R,0.507,0.858,5,-6.156,0,0.2850,0.234000,0.000000,0.1450,0.591,97.682,4,Raw Expression,Brukout edition: It's a raw and sexy dancehall...,42,307
88621,3yUO32lGCEnAfFGORSEsy4,0K22864F8jwAymy5JsopR9,Touch Yah,156708,['06JH3wcXkJg7eoxZVMO1UX'],['T.O.K'],2HOMRFOtWZrMdHkGqyuQJR,D-Stress Riddim,0,0K22864F8jwAymy5JsopR9,0.634,0.811,0,-4.661,0,0.3370,0.399000,0.000000,0.3030,0.794,99.644,4,Raw Expression,Brukout edition: It's a raw and sexy dancehall...,42,307


In [15]:
pd_full = pd_full.drop('playlist_name', axis = 1)
pd_full = pd_full.drop('playlist_description', axis = 1)
pd_full = pd_full.drop('id', axis = 1)

In [16]:
pd_full

,playlist_id,track_id,track_name,track_duration,artist_ids,artist_names,album_id,album_name,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,playlist_num_tracks,playlist_num_followers
0,37i9dQZF1DXcBWIGoYBM5M,5HCyWlXZPP0y6Gqq8TgA20,STAY (with Justin Bieber),141805,"['2tIP7SsRs7vjIcLrU85W8J', '1uNFoZAHBGtllmzznp...","['The Kid LAROI', 'Justin Bieber']",4QLAtpLNUsHEYrcHXmMIZZ,STAY (with Justin Bieber),100,0.591,0.764,1,-5.484,1,0.0483,0.038300,0.000000,0.1030,0.478,169.928,4,50,29379379
1,37i9dQZF1DXcBWIGoYBM5M,27NovPIUIRrOZoCHxABJwK,INDUSTRY BABY (feat. Jack Harlow),212000,"['7jVv8c5Fj3E9VhNjxT4snq', '2LIk90788K0zvyj2JJ...","['Lil Nas X', 'Jack Harlow']",622NFw5Yk0OReMJ2XWcXUh,INDUSTRY BABY (feat. Jack Harlow),97,0.736,0.704,3,-7.409,0,0.0615,0.020300,0.000000,0.0501,0.894,149.995,4,50,29379379
2,37i9dQZF1DXcBWIGoYBM5M,3Vi5XqYrmQgOYBajMWSvCi,Need To Know,210560,['5cj0lLjcoR7YOSnhnX0Po5'],['Doja Cat'],1nAQbHeOWTfQzbOoFrvndW,Planet Her,92,0.664,0.609,1,-6.509,1,0.0707,0.304000,0.000000,0.0926,0.194,130.041,4,50,29379379
3,37i9dQZF1DXcBWIGoYBM5M,6PQ88X9TkUIAUIZJHW2upE,Bad Habits,231041,['6eUKZXaKkcviH0Ku9w2n3V'],['Ed Sheeran'],01NhUvviMytvV12pmJiDZH,Bad Habits,98,0.808,0.897,11,-3.712,0,0.0348,0.046900,0.000031,0.3640,0.591,126.026,4,50,29379379
4,37i9dQZF1DXcBWIGoYBM5M,3Wrjm47oTz2sjIgck11l5e,Beggin',211560,['0lAWpj5szCSwM4rUMHYmrr'],['Måneskin'],2qJw6w5XwQO0PQlSWPu7Tw,Chosen,99,0.714,0.800,11,-4.808,0,0.0504,0.127000,0.000000,0.3590,0.589,134.002,4,50,29379379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88618,3yUO32lGCEnAfFGORSEsy4,6jKNYORXeWGGGiSkj1sBZu,Cocky Tuff Up,170111,['5mYWkDD4b1eM4ZjFq5axxs'],['Aidonia'],4iSfhNdg4CcSoHBZ8MNR0S,Matte Black Riddim,0,0.776,0.924,7,-4.651,1,0.0580,0.243000,0.000075,0.1580,0.749,104.975,4,42,307
88619,3yUO32lGCEnAfFGORSEsy4,5DycXVa4e82E6k9M15Nf51,Turn It Up (feat. DJ Jazzy T),220978,"['1RnGhd2JfN5nbVOvYmhDyO', '4DxQlFDzfkQjyU9mrq...","[""Ce'Cile"", 'DJ Jazzy T']",0ZEmsBG911uKT7CJxlF0DS,Still Running,6,0.646,0.947,7,-3.631,1,0.0772,0.000389,0.000057,0.8960,0.793,100.027,4,42,307
88620,3yUO32lGCEnAfFGORSEsy4,41gA2hLHRJW4bC40lm2X7R,Cup Yah Full,227552,['21J3YJTyq1biE3SvSNjzuf'],['Capleton'],1GIhVkizUAMw7Zci6DMPdG,Cup Yah Full - Single,0,0.507,0.858,5,-6.156,0,0.2850,0.234000,0.000000,0.1450,0.591,97.682,4,42,307
88621,3yUO32lGCEnAfFGORSEsy4,0K22864F8jwAymy5JsopR9,Touch Yah,156708,['06JH3wcXkJg7eoxZVMO1UX'],['T.O.K'],2HOMRFOtWZrMdHkGqyuQJR,D-Stress Riddim,0,0.634,0.811,0,-4.661,0,0.3370,0.399000,0.000000,0.3030,0.794,99.644,4,42,307


In [17]:
pd_full = pd_full.drop_duplicates()
pd_full = pd_full.fillna('None')
pd_full

,playlist_id,track_id,track_name,track_duration,artist_ids,artist_names,album_id,album_name,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,playlist_num_tracks,playlist_num_followers
0,37i9dQZF1DXcBWIGoYBM5M,5HCyWlXZPP0y6Gqq8TgA20,STAY (with Justin Bieber),141805,"['2tIP7SsRs7vjIcLrU85W8J', '1uNFoZAHBGtllmzznp...","['The Kid LAROI', 'Justin Bieber']",4QLAtpLNUsHEYrcHXmMIZZ,STAY (with Justin Bieber),100,0.591,0.764,1,-5.484,1,0.0483,0.038300,0.000000,0.1030,0.478,169.928,4,50,29379379
1,37i9dQZF1DXcBWIGoYBM5M,27NovPIUIRrOZoCHxABJwK,INDUSTRY BABY (feat. Jack Harlow),212000,"['7jVv8c5Fj3E9VhNjxT4snq', '2LIk90788K0zvyj2JJ...","['Lil Nas X', 'Jack Harlow']",622NFw5Yk0OReMJ2XWcXUh,INDUSTRY BABY (feat. Jack Harlow),97,0.736,0.704,3,-7.409,0,0.0615,0.020300,0.000000,0.0501,0.894,149.995,4,50,29379379
2,37i9dQZF1DXcBWIGoYBM5M,3Vi5XqYrmQgOYBajMWSvCi,Need To Know,210560,['5cj0lLjcoR7YOSnhnX0Po5'],['Doja Cat'],1nAQbHeOWTfQzbOoFrvndW,Planet Her,92,0.664,0.609,1,-6.509,1,0.0707,0.304000,0.000000,0.0926,0.194,130.041,4,50,29379379
3,37i9dQZF1DXcBWIGoYBM5M,6PQ88X9TkUIAUIZJHW2upE,Bad Habits,231041,['6eUKZXaKkcviH0Ku9w2n3V'],['Ed Sheeran'],01NhUvviMytvV12pmJiDZH,Bad Habits,98,0.808,0.897,11,-3.712,0,0.0348,0.046900,0.000031,0.3640,0.591,126.026,4,50,29379379
4,37i9dQZF1DXcBWIGoYBM5M,3Wrjm47oTz2sjIgck11l5e,Beggin',211560,['0lAWpj5szCSwM4rUMHYmrr'],['Måneskin'],2qJw6w5XwQO0PQlSWPu7Tw,Chosen,99,0.714,0.800,11,-4.808,0,0.0504,0.127000,0.000000,0.3590,0.589,134.002,4,50,29379379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88618,3yUO32lGCEnAfFGORSEsy4,6jKNYORXeWGGGiSkj1sBZu,Cocky Tuff Up,170111,['5mYWkDD4b1eM4ZjFq5axxs'],['Aidonia'],4iSfhNdg4CcSoHBZ8MNR0S,Matte Black Riddim,0,0.776,0.924,7,-4.651,1,0.0580,0.243000,0.000075,0.1580,0.749,104.975,4,42,307
88619,3yUO32lGCEnAfFGORSEsy4,5DycXVa4e82E6k9M15Nf51,Turn It Up (feat. DJ Jazzy T),220978,"['1RnGhd2JfN5nbVOvYmhDyO', '4DxQlFDzfkQjyU9mrq...","[""Ce'Cile"", 'DJ Jazzy T']",0ZEmsBG911uKT7CJxlF0DS,Still Running,6,0.646,0.947,7,-3.631,1,0.0772,0.000389,0.000057,0.8960,0.793,100.027,4,42,307
88620,3yUO32lGCEnAfFGORSEsy4,41gA2hLHRJW4bC40lm2X7R,Cup Yah Full,227552,['21J3YJTyq1biE3SvSNjzuf'],['Capleton'],1GIhVkizUAMw7Zci6DMPdG,Cup Yah Full - Single,0,0.507,0.858,5,-6.156,0,0.2850,0.234000,0.000000,0.1450,0.591,97.682,4,42,307
88621,3yUO32lGCEnAfFGORSEsy4,0K22864F8jwAymy5JsopR9,Touch Yah,156708,['06JH3wcXkJg7eoxZVMO1UX'],['T.O.K'],2HOMRFOtWZrMdHkGqyuQJR,D-Stress Riddim,0,0.634,0.811,0,-4.661,0,0.3370,0.399000,0.000000,0.3030,0.794,99.644,4,42,307


## Ý tưởng thuật toán
+ Clustering tất cả các track trong tập dữ liệu
+ Trả về cluster có nhiều seed track nhất, và trả về các track trong cái cluster đó (ưu tiên trả về track có popularity cao hơn)

## Xây dựng dữ liệu để test
+ Chọn ngẫu nhiên 100 `playlist_id` để tạo một dictionary trong đó key là `playlist_id` và value là list chứa các `track_id` có trong playlist đó
+ Xây dựng tập test bằng cách bỏ đi một vài `track_id` trong list của `playlist_id` dựa vào `missing_rate`

In [18]:
playlist_test_size = 100

playlist_test_all_track = generate_playlist(pd_full, pd_playlist, num_playlist_to_test = playlist_test_size)
playlist_test_all_track

{'45Cyxm3R3Gpn2upYrjcPbc': ['3uBbvynFWlF041WzNVHd1d',
  '6LGwYMXXgURfaequXipzHx',
  '2tOZtdbkAMQASWokLvSlzj',
  '79ek2oolIYyGUO4aVK5IY2',
  '4FHu9bOzBjZurx89CMa42L',
  '5XJJdNPkwmbUwE79gv0NxK',
  '2VysjemU93ac5oRg4MfYGp',
  '53gcDgFyqqpvCzXWrnFZpH',
  '5M8nEsWqUkwUwCp58ydE4W',
  '2FgkwIXC52sHCbqVL8jE41',
  '6mxyfdyF4LD7PctU3UNyjF',
  '2Vc78Iazsizj2HwUHd8wgK',
  '7sZCAHP2duHwr5M5K7lHsb',
  '27dUo79lgLEy4Rk5w3UvSL',
  '0v3FPUiZAzViGUtXzdpB5O',
  '7l7tTR8FVBemCeJBrzBOXm',
  '6gw0VbU7mFGCFnpKS0CbYU',
  '5fkBCtLEXB6ApKFly8AtBO',
  '4UtUYddZgey5Y8sxqhW9mH',
  '7tuQOFadLOvJ7Dx0MJkVKt',
  '232luC18IyiydzScZ5LjS6',
  '2rT6jwkyUcz6cmDWZGbGjC',
  '7CGws8BSQ8v1HmjEm97XQB',
  '25ocTlbdByPTuaQQaNMlO3',
  '3EQ3P4NBEVDr2TRjaWXGnX',
  '65rRB2mspD309xE6YimZTl',
  '6vegnfDS8DAEaCqWaPYGPy',
  '7ptxlO1s34uXTPnm2S0Tno',
  '4Li2WHPkuyCdtmokzW2007',
  '4ksRwIt4WVW98orafdfjXJ',
  '2RUFzxyUlI36KKy9Agkqze',
  '6Ln2tBK4EnNtMLtYyAZhr7',
  '2jdX3ZYoTsg3LRechdSQzG',
  '1b4i9BV94aJSkoGwPSol1X',
  '3DTy1olOsyxfLmgcym1

In [19]:
missing_rate = 0.2
playlist_to_test = generate_test_playlist(playlist_test_all_track, missing_rate)
playlist_to_test

{'45Cyxm3R3Gpn2upYrjcPbc': ['3uBbvynFWlF041WzNVHd1d',
  '6LGwYMXXgURfaequXipzHx',
  '2tOZtdbkAMQASWokLvSlzj',
  '79ek2oolIYyGUO4aVK5IY2',
  '4FHu9bOzBjZurx89CMa42L',
  '5XJJdNPkwmbUwE79gv0NxK',
  '2VysjemU93ac5oRg4MfYGp',
  '53gcDgFyqqpvCzXWrnFZpH',
  '5M8nEsWqUkwUwCp58ydE4W',
  '2FgkwIXC52sHCbqVL8jE41',
  '6mxyfdyF4LD7PctU3UNyjF',
  '2Vc78Iazsizj2HwUHd8wgK',
  '7sZCAHP2duHwr5M5K7lHsb'],
 '37i9dQZF1DWX16Q183eyvd': ['0weAUscowxeqDtpCgtbpgp',
  '3QpAo1aiTUVgsQBR8KmNHO',
  '1JIICzNjNibNpB3HeefvFm',
  '6ZSMbNa2ZSmLR7LlC5Bz0m',
  '00Nx5kFUJBcdGa0GNPH8Lx',
  '6l8PVMHkToebBTlJacW6Fa',
  '3vyExFn4NbOCGkGR30i6AA',
  '0PylWiOSendDTztmimb42y',
  '2b9sTGqIWVTFVu7AUMQn7h',
  '3xDiCJdE8eRRgx0qUOQphX'],
 '37i9dQZF1DWWYlokKRoQoC': ['5jYq5kj7kt6Iokzv7XeZhZ',
  '7IijX5DkLaf3G4qF6xDLYa',
  '57M2Tzakrrj4pecyVet6kM',
  '7zrxGPR1UVK2iSK793vLPl',
  '5xIVA72uBH9dsTNC7L37LV',
  '511dXUrzAfeivY8f0LUMZd',
  '0XP9ngwJpoFXVEeA7xcKHU',
  '4cwuAYkhbb775IejX7YgGn',
  '00NJC5H7lMteO234u8BvcN',
  '0YqwZcztnGN3Rjh4yPGmi

In [20]:
pd_full_test = pd_full.copy()
playlist_to_test_id = list(playlist_to_test.keys()) 
pd_full_test = pd_full_test[pd_full_test['playlist_id'].isin(playlist_to_test_id)]
pd_full_test

,playlist_id,track_id,track_name,track_duration,artist_ids,artist_names,album_id,album_name,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,playlist_num_tracks,playlist_num_followers
150,37i9dQZF1DXc2aPBXGmXrt,5HCyWlXZPP0y6Gqq8TgA20,STAY (with Justin Bieber),141805,"['2tIP7SsRs7vjIcLrU85W8J', '1uNFoZAHBGtllmzznp...","['The Kid LAROI', 'Justin Bieber']",4QLAtpLNUsHEYrcHXmMIZZ,STAY (with Justin Bieber),100,0.591,0.764,1,-5.484,1,0.0483,0.038300,0.000000,0.1030,0.478,169.928,4,69,2365211
151,37i9dQZF1DXc2aPBXGmXrt,4iJyoBOLtHqaGxP12qzhQI,Peaches (feat. Daniel Caesar & Giveon),198081,"['1uNFoZAHBGtllmzznpCI3s', '20wkVLutqVOYrc0kxF...","['Justin Bieber', 'Daniel Caesar', 'Giveon']",5dGWwsZ9iB2Xc3UKR0gif2,Justice,94,0.677,0.696,0,-6.181,1,0.1190,0.321000,0.000000,0.4200,0.464,90.030,4,69,2365211
152,37i9dQZF1DXc2aPBXGmXrt,1mSdbey7RstGLY2udgXv74,Essence (feat. Justin Bieber & Tems),263288,"['3tVQdUvClmAT7URs9V3rsp', '1uNFoZAHBGtllmzznp...","['WizKid', 'Justin Bieber', 'Tems']",0Rn8NAh8NnoaCOt8r8Tc3e,Essence (feat. Justin Bieber & Tems),77,0.835,0.691,0,-5.828,1,0.1130,0.015900,0.000014,0.1080,0.581,103.999,4,69,2365211
153,37i9dQZF1DXc2aPBXGmXrt,55sgQENF3LY1sUcJUwasLl,Don’t Go (with Justin Bieber & Don Toliver),168253,"['5he5w2lnU9x7JFhnwcekXX', '1uNFoZAHBGtllmzznp...","['Skrillex', 'Justin Bieber', 'Don Toliver']",76HLs4wSTOuCNwcGl4dycr,Don’t Go (with Justin Bieber & Don Toliver),75,0.689,0.695,11,-5.220,1,0.0429,0.019000,0.000000,0.1230,0.359,75.050,4,69,2365211
154,37i9dQZF1DXc2aPBXGmXrt,2wrJq5XKLnmhRXHIAf9xBa,"10,000 Hours (with Justin Bieber)",167693,"['7z5WFjZAIYejWy0NI5lv4T', '1uNFoZAHBGtllmzznp...","['Dan + Shay', 'Justin Bieber']",4ow6xJwn49gpWz7iHpOzWY,"10,000 Hours (with Justin Bieber)",82,0.654,0.630,10,-4.644,1,0.0259,0.153000,0.000000,0.1110,0.430,89.991,4,69,2365211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88481,4muFXNJjmJvLWdHmx0PKYO,0D256tN7htrqjbPJeaU5Dj,Let Us Stay Young (Lucas Nord vs. Urban Cone) ...,204912,"['4ZUg3IUvAPAl8coXQAxaXd', '3WOOglGBDGvr6c2WBe...","['Lucas Nord', 'Urban Cone']",03O5hpPHEZwywRMNpS5nmY,Let Us Stay Young (Lucas Nord vs. Urban Cone),0,0.582,0.750,6,-4.104,1,0.0362,0.089500,0.000042,0.0875,0.322,128.026,4,48,36
88482,4muFXNJjmJvLWdHmx0PKYO,6gLTOfcCNzugIDv6XAkjYU,Curtain Call,162354,['30fJxDFMqv33K97dUEoFeK'],['Simian Ghost'],5Pfecd5UlXCBSGua8KYvMd,Youth,2,0.507,0.642,7,-6.171,1,0.0336,0.100000,0.000000,0.0754,0.636,125.010,4,48,36
88483,4muFXNJjmJvLWdHmx0PKYO,08nJLlYp7OmDmwB7TzeX7I,Ghost In My Head,150769,['3BvEQzdNFTD0hoeZCAt3WZ'],['Niva'],3HQA58tq0MSmNdXQDBQmQw,Feverish Dreams,0,0.526,0.915,8,-6.255,0,0.0390,0.259000,0.842000,0.0816,0.709,156.000,4,48,36
88484,4muFXNJjmJvLWdHmx0PKYO,2daYRTZX7lRtQ6AjeAaNct,Last Kiss,295524,['2aS4IlhOL11IDPtN2KFDse'],['LCTRISC'],150rc0fH2PMzgISllNS26F,Last Kiss,0,0.752,0.799,9,-5.868,0,0.0719,0.023500,0.177000,0.1390,0.544,132.017,4,48,36


In [21]:
columns_to_drop = ['playlist_id', 'track_id', 'track_name', 'artist_ids', 'artist_names', 'album_id', 'album_name', 
                   'playlist_num_tracks', 'playlist_num_followers']
pd_full_test = pd_full_test.drop(columns_to_drop, axis = 1)
pd_full_test

,track_duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
150,141805,100,0.591,0.764,1,-5.484,1,0.0483,0.038300,0.000000,0.1030,0.478,169.928,4
151,198081,94,0.677,0.696,0,-6.181,1,0.1190,0.321000,0.000000,0.4200,0.464,90.030,4
152,263288,77,0.835,0.691,0,-5.828,1,0.1130,0.015900,0.000014,0.1080,0.581,103.999,4
153,168253,75,0.689,0.695,11,-5.220,1,0.0429,0.019000,0.000000,0.1230,0.359,75.050,4
154,167693,82,0.654,0.630,10,-4.644,1,0.0259,0.153000,0.000000,0.1110,0.430,89.991,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88481,204912,0,0.582,0.750,6,-4.104,1,0.0362,0.089500,0.000042,0.0875,0.322,128.026,4
88482,162354,2,0.507,0.642,7,-6.171,1,0.0336,0.100000,0.000000,0.0754,0.636,125.010,4
88483,150769,0,0.526,0.915,8,-6.255,0,0.0390,0.259000,0.842000,0.0816,0.709,156.000,4
88484,295524,0,0.752,0.799,9,-5.868,0,0.0719,0.023500,0.177000,0.1390,0.544,132.017,4


In [22]:
sum = 0
for key in playlist_test_all_track.keys():
    sum += len(playlist_test_all_track[key])
assert sum == pd_full_test.shape[0]

In [23]:
pd_full_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5649 entries, 150 to 88485
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_duration    5649 non-null   int64  
 1   popularity        5649 non-null   int64  
 2   danceability      5649 non-null   float64
 3   energy            5649 non-null   float64
 4   key               5649 non-null   int64  
 5   loudness          5649 non-null   float64
 6   mode              5649 non-null   int64  
 7   speechiness       5649 non-null   float64
 8   acousticness      5649 non-null   float64
 9   instrumentalness  5649 non-null   float64
 10  liveness          5649 non-null   float64
 11  valence           5649 non-null   float64
 12  tempo             5649 non-null   float64
 13  time_signature    5649 non-null   int64  
dtypes: float64(9), int64(5)
memory usage: 662.0 KB


## Tiến hành chuẩn hóa và xây dựng mô hình
+ Sử dụng MinMaxScaler() để chuẩn hóa dữ liệu
+ Xây dựng K-Means
+ Khảo sát số lượng `k` dựa vào elbow method và chọn số `k` tốt nhất

In [27]:
scaler = MinMaxScaler()
scaler.fit(pd_full_test)
scaled_pd_full_test = scaler.transform(pd_full_test)